# SIMULATOR

## Equation

The block reward function is;

$R_{ij} = log_{10} (S_{ij}) \times log_{10}(D_j) \times R_i$

Where;

$i$ is the actor

$j$ is the dataset

$S$ is the stake in **drops**

$D$ is the number of deliveries of the dataset or service

$R_i$ is the Sybil download factor; 


The Sybil download factor penalizes actors who download thier data repeatedly to increase their reward; 

$R_i = [min(B_{served}, B_{downloaded}, 1.0]$ if all data assets served,

$R_i = 0 $ otherwise

ERROR IN EQUATION?? 


In [1]:
import logging
#Delete Jupyter notebook root logger handler
logger = logging.getLogger()
logger.handlers = []

# Set level
logger.setLevel(logging.DEBUG)

# Create formatter
#FORMAT = "%(asctime)s - %(levelno)-3s - %(module)-10s  %(funcName)-10s: %(message)s"
FORMAT = "%(asctime)s - %(levelno)-3s - %(funcName)-10s: %(message)s"
formatter = logging.Formatter(FORMAT)

# Create handler and assign
handler = logging.StreamHandler(sys.stderr)
handler.setFormatter(formatter)
logger.handlers = [handler]
logging.debug("Logging started")

2018-06-08 10:48:04,328 - 10  - <module>  : Logging started
2018-06-08 10:48:04,335 - 10  - pylab_setup: backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
import numpy as np
from secrets import randbits
from collections import defaultdict
#hex(randbits(256))

In [3]:
def reward():
    return np.log10(stake) * np.log10(number_deliveries)

### Actors (off-chain roles)

In [4]:
class Actor():
    def __init__(self,name, capital = 0):
        self.name = name
        self.wallet_address = hex(randbits(256))
        self.capital = capital
        print(self)
    def __str__(self):
        return "{} with {} EUR".format(self.name,self.capital)
    

In [5]:
a_bob = Actor("Bob", 10000)
a_alice = Actor("Alice", 500)
a_bigbrother = Actor("Big Brother", 3000000)
a_housing = Actor("Housing", -250000)
a_luigi = Actor("Luigi", 125000)

Bob with 10000 EUR
Alice with 500 EUR
Big Brother with 3000000 EUR
Housing with -250000 EUR
Luigi with 125000 EUR


### Datasets (off-chain)

In [6]:
class DataSet():
    def __init__(self, name, description, size, url = "/SOME URL"):
        self.name = name
        self.description = description
        self.size = size # bits
        self.url = url
        self.id = hex(randbits(256))
        print(self)
    def __str__(self):
        return "{} dataset: {}, {} MB".format(self.name,self.description, self.size/1024/1024)
    
    #def create_dataset(self, data_name):
    #    self.data_name = data_name

In [7]:
ds_audio_files = DataSet("audio_ryerson", "actor emotional speech data", 123 * 1024 * 1024)
ds_financial = DataSet("finance_bitcoin", "time series bitcoin prices", 1500 * 1024 * 1024)
ds_pidgeons = DataSet("pidgeon_race","Pidgeon racing track and features", 112 * 1024)
ds_drone_image = DataSet("Drone_image","Personal Customer Image Data", 5 * 1024 * 1024 * 1024)
ds_drone_transactions = DataSet("Drone_transactions","Customer Transaction History", 10 * 1024 * 1024)

audio_ryerson dataset: actor emotional speech data, 123.0 MB
finance_bitcoin dataset: time series bitcoin prices, 1500.0 MB
pidgeon_race dataset: Pidgeon racing track and features, 0.109375 MB
Drone_image dataset: Personal Customer Image Data, 5120.0 MB
Drone_transactions dataset: Customer Transaction History, 10.0 MB


### The MARKET (on-chain)

In [8]:
class Asset():
    """
    Represent the Asset structure in Solidity
    """
    def __init__(self,SECRET_name,SECRET_dataset_id,owner,ndrops,bit_size,url,token,active):
        self.SECRET_name = SECRET_name
        self.SECRET_dataset_id = SECRET_dataset_id
        self.owner   = owner                       # address - owner of the Asset 
        self.ndrops  = ndrops                       # uint256 - total supply of Drops 
        self.bitSize  = bit_size                      # uint256 - size of asset in bit 
        self.url     = url                       # bytes32 - url of the asset 
        self.token   = token                       # bytes32 - token to get access to the asset 
        self.active  = active                        # bool  
    
    def __str__(self):
        return "[{}] owned by {}, {} drops, {} MB, at {}, token {}, active: {} ".format(
            self.SECRET_name, 
                                                                self.owner[-4:], 
                                                                self.ndrops,
                                                                self.bitSize/1024/1024,
                                                                self.url,
                                                                self.token,
                                                                self.active,
                                                               )

In [9]:
class Provider():
    """
    Represent the Provider structure in Solidity
    """
    def __init__(self,SECRET_name, provider,numOCN,allowanceOCN,uploadBits,downloadBits):
        self.SECRET_name = SECRET_name
        self.provider   = provider                       # address
        self.numOCN  = numOCN                       # uint256 - Ocean token balance
        self.allowanceOCN  = allowanceOCN                      # available Ocean tokens for transfer excuding locked tokens for staking 
        self.uploadBits     = uploadBits                       # uint256 - total number of bits that served across all data assets with stakes
        self.downloadBits   = downloadBits                       # uint256 - total number of bits that download across all data assets with stakes

    def __str__(self):
        return "[{}] with {} MB uploaded, {} MB downloaded, {} OCB".format(
            self.SECRET_name,
            self.uploadBits/1024/1024,
            self.downloadBits/1024/1024,
            self.numOCN,
                                                               )

In [10]:
class Market():
    def __init__(self,start,stop):
        self.tick = start
        self.stop = stop
        self.mProviders = dict()
        self.mAssets = dict()

    def __iter__(self):
        return self        

    def next(self): # Python 3: def __next__(self)
        if self.tick > self.stop:
            raise StopIteration
        else:
            self.tick += 1
            return self.tick
        
    def __str__(self):
        return "Tick {}, {} assets, {} providers".format(self.tick, 
                                           len(self.mAssets),
                                           len(self.mProviders),
                                          )
    def register(self, sender, dataset):
        """ 
        Convert a dataset to an Asset, register it in the market
        Register the Provider, if not already present
        """
        asset_id = hex(randbits(256))
                       
        if sender.wallet_address not in self.mProviders.keys():
            self.mProviders[sender.wallet_address] = Provider(sender.name,sender.wallet_address, 0, 0, 0, 0)
            logging.info("[{}] registered as a new provider".format(sender.name))
        
        
        self.mProviders[sender.wallet_address].uploadBits += dataset.size

        this_asset = Asset(SECRET_name = dataset.name,
                           SECRET_dataset_id = dataset.id,
                            owner = sender.wallet_address,
                           ndrops = 0,
                           bit_size = dataset.size,
                           url = dataset.url,
                           token = None,
                           active = False,
                          )
        
        self.mAssets[asset_id] = this_asset
        logging.info("[{}] registered {} MB {} dataset into market".format(sender.name,dataset.size/1000/1000,dataset.name))

    def publish(self):
        pass
    
    def purchase(self):
        pass
    

    #def 

In [11]:
market = Market(0,1000)
print(market)

Tick 0, 0 assets, 0 providers


In [12]:
market.register(a_bob, ds_financial)
market.register(a_bob, ds_pidgeons)
market.register(a_alice, ds_audio_files)
market.register(a_luigi, ds_drone_image)
market.register(a_luigi, ds_drone_transactions)

2018-06-08 10:48:04,390 - 20  - register  : [Bob] registered as a new provider
2018-06-08 10:48:04,391 - 20  - register  : [Bob] registered 1572.864 MB finance_bitcoin dataset into market
2018-06-08 10:48:04,391 - 20  - register  : [Bob] registered 0.114688 MB pidgeon_race dataset into market
2018-06-08 10:48:04,392 - 20  - register  : [Alice] registered as a new provider
2018-06-08 10:48:04,392 - 20  - register  : [Alice] registered 128.974848 MB audio_ryerson dataset into market
2018-06-08 10:48:04,392 - 20  - register  : [Luigi] registered as a new provider
2018-06-08 10:48:04,393 - 20  - register  : [Luigi] registered 5368.70912 MB Drone_image dataset into market
2018-06-08 10:48:04,393 - 20  - register  : [Luigi] registered 10.48576 MB Drone_transactions dataset into market


In [13]:
market.next()
print(market)

Tick 1, 5 assets, 3 providers


## Market information and [Oracle information]

In [14]:
for provider in market.mProviders:
    print(market.mProviders[provider])

[Bob] with 1500.109375 MB uploaded, 0.0 MB downloaded, 0 OCB
[Alice] with 123.0 MB uploaded, 0.0 MB downloaded, 0 OCB
[Luigi] with 5130.0 MB uploaded, 0.0 MB downloaded, 0 OCB


In [15]:
asset_list = defaultdict(list)
for asset in market.mAssets:
    owner_id = market.mAssets[asset].owner
    asset_list[market.mProviders[owner_id].SECRET_name].append(market.mAssets[asset])

for provider in asset_list:
    #print(market.mProviders[provider])    
    print("{}'s assets in the market:".format(provider))
    for asset in asset_list[provider]:
        print("\t",asset)

Bob's assets in the market:
	 [finance_bitcoin] owned by f3dc, 0 drops, 1500.0 MB, at /SOME URL, token None, active: False 
	 [pidgeon_race] owned by f3dc, 0 drops, 0.109375 MB, at /SOME URL, token None, active: False 
Alice's assets in the market:
	 [audio_ryerson] owned by dc0b, 0 drops, 123.0 MB, at /SOME URL, token None, active: False 
Luigi's assets in the market:
	 [Drone_image] owned by 7226, 0 drops, 5120.0 MB, at /SOME URL, token None, active: False 
	 [Drone_transactions] owned by 7226, 0 drops, 10.0 MB, at /SOME URL, token None, active: False 
